In [1]:
!pip install pandas==1.3.2

In [2]:
!pip install swifter
!pip install Sastrawi

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
import csv
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
df = pd.read_csv(r'polemik.csv')
df.head()

,Unnamed: 0,Komentar,Label
0,1,Lha kekerasan seksual itu kan mmg definisinya ...,positif
1,2,Jangan mendekati zina. Itu adalah aturan yang ...,negatif
2,3,The law is an art. Semakin rinci semakin mudah...,positif
3,4,TETAPKAN DULU APA YANG DIMAKSUD DENGAN KEKERAS...,netral
4,5,Lbih cock diterapkan dikemendikbud Dan DPR.,netral


In [3]:
del df['Unnamed: 0']
df.head()

,Komentar,Label
0,Lha kekerasan seksual itu kan mmg definisinya ...,positif
1,Jangan mendekati zina. Itu adalah aturan yang ...,negatif
2,The law is an art. Semakin rinci semakin mudah...,positif
3,TETAPKAN DULU APA YANG DIMAKSUD DENGAN KEKERAS...,netral
4,Lbih cock diterapkan dikemendikbud Dan DPR.,netral


In [4]:
#slang/Slang word
import re
import nltk
import pandas as pd


def casefolding(review):
    review = review.lower()
    return review

def tokenize(review):
    token = nltk.word_tokenize(review)
    return token

def filtering(review):
    # Remove angka termasuk angka yang berada dalam string
    # Remove non ASCII chars
    review = re.sub(r'[^\x00-\x7f]', r'', review)
    review = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', review)
    review = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", review)
    review = re.sub(r'\\u\w\w\w\w', '', review)
    # Remove link web
    review = re.sub(r'http\S+', '', review)
    # Remove @username
    review = re.sub('@[^\s]+', '', review)
    # Remove #tagger
    review = re.sub(r'#([^\s]+)', '', review)
    # Remove simbol, angka dan karakter aneh
    review = re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", review)
    return review

def replaceThreeOrMore(review):
    # Pattern to look for three or more repetitions of any character, including newlines (contoh goool -> gol).
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1", review)

def convertToSlangword(review):
    kamus_slangword = eval(open("combined_slang_words.txt").read()) # Membuka dictionary slangword
    pattern = re.compile(r'\b( ' + '|'.join (kamus_slangword.keys())+r')\b') # Search pola kata (contoh kpn -> kapan)
    content = []
    for kata in review:
        filteredSlang = pattern.sub(lambda x: kamus_slangword[x.group()],kata) # Replace slangword berdasarkan pola review yg telah ditentukan
        content.append(filteredSlang.lower())
    review = content
    return review

def removeStopword(review):
    stopwords = open('combined_stop_words.txt', 'r').read().split()
    content = []
    filteredtext = [word for word in review.split() if word not in stopwords]
    content.append(" ".join(filteredtext))
    review = content
    return review

# stringData = casefolding(String)
# print (stringData)

data = df
# Keeping only the neccessary columns
datasets = [data]
   
for teks in datasets:
    label = teks['Label']
    teks = teks['Komentar'].apply(casefolding)
    teks = teks.apply(filtering)
    teks = teks.apply(replaceThreeOrMore)
    teks = teks.apply(tokenize)
    teks = teks.apply(convertToSlangword)
    teks = teks.apply(" ".join)
    teks = teks.apply(removeStopword)
    teks = teks.apply(" ".join)
    print(teks)

review_dict = {'Komentar': teks, 'Label' : label}
df = pd.DataFrame(review_dict, columns = ['Komentar', 'Label'])
print(df.info())
df.to_csv('data-bersih.csv', sep= ',' , encoding='utf-8')

0      kekerasan seksual kan memang definisinya meman...
1      mendekati zina aturan pasal dl sat demokrasi t...
2                    law is an art rinci mudah dibelokan
3                         tetapkan kekerasan seksualitas
4                     cocok diterapkan dikemendikbud dpr
                             ...                        
641    udah ganti menterinya sekolah pendidikan ambur...
642                                   hukum mati selesai
643                                  laki laki perempuan
644                                                germo
645    gawat darurat dikampus karenahukum jalan nadim...
Name: Komentar, Length: 646, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Komentar  646 non-null    object
 1   Label     646 non-null    object
dtypes: object(2)
memory usage: 10.2+ KB
None


In [5]:
#cleaning Text
def remove_punct(text):
    text = re.sub(r'[^a-zA-Z0-9]', ' ', str(text))
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = re.sub(r'\b\w(1,2)\b', '', text)
    text = re.sub(r'\s\s+', ' ', text)
    text = re.sub(r"\d+", "", text)
    return text
df['Komentar'] = df['Komentar'].apply(remove_punct) 
df.head()

,Komentar,Label
0,kekerasan seksual kan memang definisinya meman...,positif
1,mendekati zina aturan pasal dl sat demokrasi t...,negatif
2,law is an art rinci mudah dibelokan,positif
3,tetapkan kekerasan seksualitas,netral
4,cocok diterapkan dikemendikbud dpr,netral


In [6]:
df.to_csv(r'cleaning Text.csv')

In [7]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df['Komentar'] = df['Komentar'].str.lower()
df.head()

,Komentar,Label
0,kekerasan seksual kan memang definisinya meman...,positif
1,mendekati zina aturan pasal dl sat demokrasi t...,negatif
2,law is an art rinci mudah dibelokan,positif
3,tetapkan kekerasan seksualitas,netral
4,cocok diterapkan dikemendikbud dpr,netral


In [8]:
df.to_csv(r'Case Folding.csv')

In [9]:
def replace_repeat_character(text):
    # Pattern to look for three or more repetitions of any character, including newlines (contoh goool -> gol).
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", text)

df['Komentar'] = df['Komentar'].apply(replace_repeat_character) 
df.head()

,Komentar,Label
0,kekerasan seksual kan memang definisinya meman...,positif
1,mendekati zina aturan pasal dl sat demokrasi t...,negatif
2,law is an art rinci mudah dibelokan,positif
3,tetapkan kekerasan seksualitas,netral
4,cocok diterapkan dikemendikbud dpr,netral


In [10]:
from nltk.tokenize import word_tokenize

def tokenize(text):
    return nltk.word_tokenize(text)
    
df['TOKENIZATION'] = df['Komentar'].apply(tokenize)
df.head(10)

,Komentar,Label,TOKENIZATION
0,kekerasan seksual kan memang definisinya meman...,positif,"[kekerasan, seksual, kan, memang, definisinya,..."
1,mendekati zina aturan pasal dl sat demokrasi t...,negatif,"[mendekati, zina, aturan, pasal, dl, sat, demo..."
2,law is an art rinci mudah dibelokan,positif,"[law, is, an, art, rinci, mudah, dibelokan]"
3,tetapkan kekerasan seksualitas,netral,"[tetapkan, kekerasan, seksualitas]"
4,cocok diterapkan dikemendikbud dpr,netral,"[cocok, diterapkan, dikemendikbud, dpr]"
5,pakaian anak sekolah mahasiswa rok nya lutut g...,negatif,"[pakaian, anak, sekolah, mahasiswa, rok, nya, ..."
6,korban ragu melapor usulan perlindungan hukum ...,positif,"[korban, ragu, melapor, usulan, perlindungan, ..."
7,presenter suka motong percakapan narasumber sadar,netral,"[presenter, suka, motong, percakapan, narasumb..."
8,ormas islam mendukung permedikbudristek kekera...,positif,"[ormas, islam, mendukung, permedikbudristek, k..."
9,negara kacau pejabatnya nyeleneh coba urus sub...,negatif,"[negara, kacau, pejabatnya, nyeleneh, coba, ur..."


In [11]:
katabaku = csv.reader(
	open(r"kata_baku.csv"), delimiter=";") # ambil file csv kata baku menjadi array

kamus_katabaku = {} # empty dictionary untuk kamus kata baku
try:
    for row in katabaku : # membuat kamus kata baku dengan input kata tidak baku dan output kata bakunya
        kamus_katabaku[row[0]] = row[1]
except:
    pass

In [12]:
print(kamus_katabaku)

{'abjad': 'abjat', 'akta': 'akte', 'aktif': 'aktiv', 'aktivitas': 'aktifitas', 'lah': 'lha', 'amendemen': 'amandemen', 'andal': 'handal', 'analisis': 'analisa', 'antre': 'antri', 'asas': 'azas', 'apak': 'apek', 'apotek': 'apotik', 'aritmetika': 'aritmatika', 'atlet': 'atlit', 'atmosfer': 'atmosfir', 'cabai': 'cabay', 'camilan': 'cemilan', 'cendekia': 'cendikia', 'cenderamata': 'cinderamata', 'detail': 'detil', 'definisi': 'difinisi', 'diagnosis': 'diagnosa', 'embus': 'hembus', 'ekstra': 'extra', 'ekstrem': 'ekstrim', 'esai': 'esei', 'februari': 'febuari', 'fondasi': 'pondasi', 'formal': 'formil', 'frasa': 'frase', 'frekuensi': 'frekwensi', 'gizi': 'gisi', 'hafal': 'hapal', 'hakikat': 'hakekat', 'hipotesis': 'hipotesa', 'hierarki': 'hirarkhi', 'ijazah': 'ijasah', 'izin': 'ijin', 'imbau': 'himbau', 'isap': 'hisap', 'istri': 'isteri', 'jadwal': 'jadual', 'jenazah': 'jenasah', 'jenderal': 'jendral', 'justru': 'justeru', 'kafetaria': 'kafeteria', 'kalender': 'kalendar', 'karier': 'karir', '

In [13]:
def normalized_term(text):
    return [kamus_katabaku[term] if term in kamus_katabaku else term for term in text]
df['TOKENIZATION'] = df['TOKENIZATION'].apply(normalized_term) 
df.head(10)

,Komentar,Label,TOKENIZATION
0,kekerasan seksual kan memang definisinya meman...,positif,"[kekerasan, seksual, kan, memang, definisinya,..."
1,mendekati zina aturan pasal dl sat demokrasi t...,negatif,"[mendekati, zinah, aturan, pasal, dl, sat, dem..."
2,law is an art rinci mudah dibelokan,positif,"[law, is, an, art, rinci, gampang, dibelokan]"
3,tetapkan kekerasan seksualitas,netral,"[tetapkan, kekerasan, seksualitas]"
4,cocok diterapkan dikemendikbud dpr,netral,"[cocok, diterapkan, dikemendikbud, dpr]"
5,pakaian anak sekolah mahasiswa rok nya lutut g...,negatif,"[pakaian, anak, sekolah, mahasiswa, rok, nya, ..."
6,korban ragu melapor usulan perlindungan hukum ...,positif,"[korban, ragu, melapor, usulan, perlindungan, ..."
7,presenter suka motong percakapan narasumber sadar,netral,"[presenter, suka, motong, percakapan, nara, sa..."
8,ormas islam mendukung permedikbudristek kekera...,positif,"[ormas, islam, mendukung, permedikbudristek, k..."
9,negara kacau pejabatnya nyeleneh coba urus sub...,negatif,"[negara, kacau, pejabatnya, nyeleneh, coba, ur..."


In [14]:
stopword =nltk.corpus.stopwords.words('indonesian')
stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                 'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                 'krn', 'nya', 'nih', 'sih', 
                 'si', 'tau', 'tuh', 'utk', 'ya','sdh',
                 'aja', 'n', 't', 'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                 '&amp', 'yah', 'njir', 'wkwk', 'haha', 'hehe', 'rs', 'yaa',
                 'nii', 'bin', 'lahh', 'hmm', 'lieur', 'njarem', 'kale',
                 'hd', 'an', 'ob', 'bro', 'ud', 'iyaa', 'guys', 'doang',
                 'doank', 'eh', 'kek', 'or', 'kah', 'eso', 'tuh', 'dh', 'dnk',
                 'mh', 'ja', 'tu', 'anu', 'ni', 'ku', 'blz', 'gy', 'skg', 'eeh',
                 'pas','tar','bc','x','dong','d'])

In [15]:
word_list=['enggak','tidak','jangan','belum']
stopword = [word for word in stopword if word not in word_list] #make a copy of the word_list

In [16]:
def remove_stopwords(text):
    return [word for word in text if word not in stopword]
    
df['STOP_REMOVAL'] = df['TOKENIZATION'].apply(remove_stopwords)
df.head(10)

,Komentar,Label,TOKENIZATION,STOP_REMOVAL
0,kekerasan seksual kan memang definisinya meman...,positif,"[kekerasan, seksual, kan, memang, definisinya,...","[kekerasan, seksual, definisinya, persetujuan,..."
1,mendekati zina aturan pasal dl sat demokrasi t...,negatif,"[mendekati, zinah, aturan, pasal, dl, sat, dem...","[mendekati, zinah, aturan, pasal, dl, sat, dem..."
2,law is an art rinci mudah dibelokan,positif,"[law, is, an, art, rinci, gampang, dibelokan]","[law, is, art, rinci, gampang, dibelokan]"
3,tetapkan kekerasan seksualitas,netral,"[tetapkan, kekerasan, seksualitas]","[tetapkan, kekerasan, seksualitas]"
4,cocok diterapkan dikemendikbud dpr,netral,"[cocok, diterapkan, dikemendikbud, dpr]","[cocok, diterapkan, dikemendikbud, dpr]"
5,pakaian anak sekolah mahasiswa rok nya lutut g...,negatif,"[pakaian, anak, sekolah, mahasiswa, rok, nya, ...","[pakaian, anak, sekolah, mahasiswa, rok, lutut..."
6,korban ragu melapor usulan perlindungan hukum ...,positif,"[korban, ragu, melapor, usulan, perlindungan, ...","[korban, ragu, melapor, usulan, perlindungan, ..."
7,presenter suka motong percakapan narasumber sadar,netral,"[presenter, suka, motong, percakapan, nara, sa...","[presenter, suka, motong, percakapan, nara, sa..."
8,ormas islam mendukung permedikbudristek kekera...,positif,"[ormas, islam, mendukung, permedikbudristek, k...","[ormas, islam, mendukung, permedikbudristek, k..."
9,negara kacau pejabatnya nyeleneh coba urus sub...,negatif,"[negara, kacau, pejabatnya, nyeleneh, coba, ur...","[negara, kacau, pejabatnya, nyeleneh, coba, ur..."


In [17]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import swifter

# implementasi pada data kita

# stemmed
def stemmed_wrapper(text):
    return stemmer.stem(text)

text_dict = {}

for document in df['STOP_REMOVAL']:
    for text in document:
        if text not in text_dict:
            text_dict[text] = ' '
            
print(len(text_dict))
print("------------------------")

for text in text_dict:
    text_dict[text] = stemmed_wrapper(text)
    print(text,":" ,text_dict[text])
    
print(text_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [text_dict[text] for text in document]


1462
------------------------
kekerasan : keras
seksual : seksual
definisinya : definisi
persetujuan : tuju
korban : korban
logika : logika
nolak : nolak
melawan : lawan
didefinisikan : definisi
mendekati : dekat
zinah : zinah
aturan : atur
pasal : pasal
dl : dl
sat : sat
demokrasi : demokrasi
tdak : tdak
layak : layak
isi : isi
kepentingan : penting
petingi : tingi
kemaslahatan : maslahat
masyarakatnya : masyarakat
jadikan : jadi
tameng : tameng
oknum : oknum
mengunakan : mengunakan
media : media
skrang : skrang
bicara : bicara
berbicara : bicara
sembarangan : sembarang
law : law
is : is
art : art
rinci : rinci
gampang : gampang
dibelokan : belok
tetapkan : tetap
seksualitas : seksualitas
cocok : cocok
diterapkan : terap
dikemendikbud : dikemendikbud
dpr : dpr
pakaian : pakai
anak : anak
sekolah : sekolah
mahasiswa : mahasiswa
rok : rok
lutut : lutut
ganti : ganti
mentri : tri
membubarkan : bubar
ragu : ragu
melapor : lapor
usulan : usul
perlindungan : lindung
hukum : hukum
polemik : 

perzinahan : perzinahan
konsensual : konsensual
terbit : terbit
dilegalisasi : legalisasi
ilegalisasi : ilegal
hashirama : hashirama
gogle : gogle
azas : azas
melarangnya : larang
legal : legal
jinah : jah
perundangan : undang
aturanya : aturanya
klu : klu
banyaknya : banyak
pelaporan : lapor
darurat : darurat
bikinlah : bikin
perendikbud : perendikbud
badan : badan
slama : slama
realita : realita
slalu : slalu
rugika : rugika
istrimu : istri
anakmu : anak
gauli : gaul
ridwan : ridwan
basuki : basuki
perselingkuhan : selingkuh
dikenakan : kena
pemerkosan : pemerkosan
prostitusi : prostitusi
dikti : dikti
kejahatan : jahat
ketentuan : tentu
dimuat : muat
verbal : verbal
non : non
penetrasi : penetrasi
pengintip : intip
body : body
shaming : shaming
grepe : grepe
hadir : hadir
penyebaran : sebar
photo : photo
vidio : vidio
pribadi : pribadi
ijin : ijin
kerab : kerab
menjadikan : jadi
terpidana : pidana
mendengar : dengar
pola : pola
fikir : fikir
politikus : politikus
golongan : golong
d

nonton : nonton
porno : porno
sidang : sidang
bwahaha : bwahaha
membela : bela
perempun : rem
mpret : mpret
urusin : urusin
kader : kader
kampanye : kampanye
menerima : terima
diskusikan : diskusi
sensasi : sensasi
jalankan : jalan
unya : unya
tindakanya : tindakanya
diamati : amat
kakehan : kakehan
jepang : jepang
aborsi : aborsi
efek : efek
diluar : luar
nikah : nikah
dilawan : lawan
dibrantas : dibrantas
tanda : tanda
jaman : jaman
naudzubilah : naudzubilah
inalilahi : inalilahi
wa : wa
roji : roji
un : un
moga : moga
oengagas : oengagas
plaku : plaku
mah : mah
ditembak : tembak
membicarakan : bicara
kode : kode
etik : etik
disesuaikan : sesuai
patokanya : patokanya
nangkepnya : nangkepnya
perbincangan : bincang
konteks : konteks
diusulkan : usul
nungu : nungu
bertambah : tambah
selese : selese
rifki : rifki
fauzi : fauzi
kelaman : kelam
giliran : gilir
ngusulin : ngusulin
dikritisi : kritisi
ginian : ginian
nyampe : nyampe
nalarnya : nalar
oposisi : oposisi
payah : payah
lelet : le

In [18]:
df['STEMMER'] = df['STOP_REMOVAL'].apply(get_stemmed_term)
df.head(10)

,Komentar,Label,TOKENIZATION,STOP_REMOVAL,STEMMER
0,kekerasan seksual kan memang definisinya meman...,positif,"[kekerasan, seksual, kan, memang, definisinya,...","[kekerasan, seksual, definisinya, persetujuan,...","[keras, seksual, definisi, tuju, korban, logik..."
1,mendekati zina aturan pasal dl sat demokrasi t...,negatif,"[mendekati, zinah, aturan, pasal, dl, sat, dem...","[mendekati, zinah, aturan, pasal, dl, sat, dem...","[dekat, zinah, atur, pasal, dl, sat, demokrasi..."
2,law is an art rinci mudah dibelokan,positif,"[law, is, an, art, rinci, gampang, dibelokan]","[law, is, art, rinci, gampang, dibelokan]","[law, is, art, rinci, gampang, belok]"
3,tetapkan kekerasan seksualitas,netral,"[tetapkan, kekerasan, seksualitas]","[tetapkan, kekerasan, seksualitas]","[tetap, keras, seksualitas]"
4,cocok diterapkan dikemendikbud dpr,netral,"[cocok, diterapkan, dikemendikbud, dpr]","[cocok, diterapkan, dikemendikbud, dpr]","[cocok, terap, dikemendikbud, dpr]"
5,pakaian anak sekolah mahasiswa rok nya lutut g...,negatif,"[pakaian, anak, sekolah, mahasiswa, rok, nya, ...","[pakaian, anak, sekolah, mahasiswa, rok, lutut...","[pakai, anak, sekolah, mahasiswa, rok, lutut, ..."
6,korban ragu melapor usulan perlindungan hukum ...,positif,"[korban, ragu, melapor, usulan, perlindungan, ...","[korban, ragu, melapor, usulan, perlindungan, ...","[korban, ragu, lapor, usul, lindung, hukum, ko..."
7,presenter suka motong percakapan narasumber sadar,netral,"[presenter, suka, motong, percakapan, nara, sa...","[presenter, suka, motong, percakapan, nara, sa...","[presenter, suka, motong, cakap, nara, sadar]"
8,ormas islam mendukung permedikbudristek kekera...,positif,"[ormas, islam, mendukung, permedikbudristek, k...","[ormas, islam, mendukung, permedikbudristek, k...","[ormas, islam, dukung, permedikbudristek, kera..."
9,negara kacau pejabatnya nyeleneh coba urus sub...,negatif,"[negara, kacau, pejabatnya, nyeleneh, coba, ur...","[negara, kacau, pejabatnya, nyeleneh, coba, ur...","[negara, kacau, jabat, nyeleneh, coba, urus, s..."


In [19]:
def fit_stemmer(text):
     text = np.array(text)
     text = ' '.join(text)
     return text

df['Komentar_Final'] = df['STEMMER'].apply(lambda x: fit_stemmer(x))
df.head(10)

,Komentar,Label,TOKENIZATION,STOP_REMOVAL,STEMMER,Komentar_Final
0,kekerasan seksual kan memang definisinya meman...,positif,"[kekerasan, seksual, kan, memang, definisinya,...","[kekerasan, seksual, definisinya, persetujuan,...","[keras, seksual, definisi, tuju, korban, logik...",keras seksual definisi tuju korban logika nola...
1,mendekati zina aturan pasal dl sat demokrasi t...,negatif,"[mendekati, zinah, aturan, pasal, dl, sat, dem...","[mendekati, zinah, aturan, pasal, dl, sat, dem...","[dekat, zinah, atur, pasal, dl, sat, demokrasi...",dekat zinah atur pasal dl sat demokrasi tdak l...
2,law is an art rinci mudah dibelokan,positif,"[law, is, an, art, rinci, gampang, dibelokan]","[law, is, art, rinci, gampang, dibelokan]","[law, is, art, rinci, gampang, belok]",law is art rinci gampang belok
3,tetapkan kekerasan seksualitas,netral,"[tetapkan, kekerasan, seksualitas]","[tetapkan, kekerasan, seksualitas]","[tetap, keras, seksualitas]",tetap keras seksualitas
4,cocok diterapkan dikemendikbud dpr,netral,"[cocok, diterapkan, dikemendikbud, dpr]","[cocok, diterapkan, dikemendikbud, dpr]","[cocok, terap, dikemendikbud, dpr]",cocok terap dikemendikbud dpr
5,pakaian anak sekolah mahasiswa rok nya lutut g...,negatif,"[pakaian, anak, sekolah, mahasiswa, rok, nya, ...","[pakaian, anak, sekolah, mahasiswa, rok, lutut...","[pakai, anak, sekolah, mahasiswa, rok, lutut, ...",pakai anak sekolah mahasiswa rok lutut ganti t...
6,korban ragu melapor usulan perlindungan hukum ...,positif,"[korban, ragu, melapor, usulan, perlindungan, ...","[korban, ragu, melapor, usulan, perlindungan, ...","[korban, ragu, lapor, usul, lindung, hukum, ko...",korban ragu lapor usul lindung hukum korban po...
7,presenter suka motong percakapan narasumber sadar,netral,"[presenter, suka, motong, percakapan, nara, sa...","[presenter, suka, motong, percakapan, nara, sa...","[presenter, suka, motong, cakap, nara, sadar]",presenter suka motong cakap nara sadar
8,ormas islam mendukung permedikbudristek kekera...,positif,"[ormas, islam, mendukung, permedikbudristek, k...","[ormas, islam, mendukung, permedikbudristek, k...","[ormas, islam, dukung, permedikbudristek, kera...",ormas islam dukung permedikbudristek keras sek...
9,negara kacau pejabatnya nyeleneh coba urus sub...,negatif,"[negara, kacau, pejabatnya, nyeleneh, coba, ur...","[negara, kacau, pejabatnya, nyeleneh, coba, ur...","[negara, kacau, jabat, nyeleneh, coba, urus, s...",negara kacau jabat nyeleneh coba urus subtantif


In [20]:
df.to_csv(r'PreProcessing.csv')

In [21]:
import pickle
pickle.dump(stopword,
            open('stopword.pkl', 'wb'),
            protocol=4)

In [22]:
import pickle
pickle.dump(kamus_katabaku,
            open('kamus_katabaku.pkl', 'wb'),
            protocol=4)